<a href="https://colab.research.google.com/github/WinLike-dev/my-test-repository/blob/main/train_place_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!python -m pip install -U albumentations
!python -m pip install opencv-python
!python -m pip install tqdm
!python -m pip install matplotlib

%cd /content/drive/MyDrive/project2/machine_learning_hw2/tfdet_classification
import sys
sys.path.append('/content/drive/MyDrive/project2/machine_learning_hw2/tfdet_classification')
print(sys.path)

import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/project2/machine_learning_hw2/tfdet_classification
['/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.12/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/project2/machine_learning_hw2/tfdet_classification', '/content/drive/MyDrive/project2/machine_learning_hw2/tfdet_classification', '/content/drive/MyDrive/project2/machine_learning_hw2/tfdet_classification', '/content/drive/MyDrive/project2/machine_learning_hw2/tfdet_classification', '/content/drive/MyDrive/project2/machine_learning_hw2/tfdet_classification', '/content/drive/MyDrive/project2/machine_learning_hw2/tfdet_classification']
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GP

In [ ]:
from lovewar_helper import place_data
from tfdet.model.classifier import place_classifier
import tensorflow as tf
import tfdet
import albumentations
from functools import partial
import numpy as np
from datetime import datetime
import os

In [ ]:
backbone_type = "vgg16"
use_pretrained_backbone = False
use_augmentation = True
learning_rate = 0.001
momentum = 0.90
epochs = 60
img_size = (512, 512, 3)
img_wh = (512, 512)
n_feature = 2048
batch_size = 16

output_name = 'vgg16_60_pretrained_x'
tr_img_root = "/content/drive/MyDrive/project2/machine_learning_hw2/love_war_place/train"
te_img_root = "/content/drive/MyDrive/project2/machine_learning_hw2/love_war_place/val"

class_dict = {
    "car": 0,
    "front_of_buliding": 1,
    "hospital": 2,
    "house": 3,
    "indoor": 4,
    "restaurant": 5,
    "rooftop": 6,
    "street": 7
}

In [ ]:
'''
1. albumentations
https://github.com/albumentations-team/albumentations

2. albumentations docs
https://albumentations.ai/docs/api_reference/augmentations/

3. recommend
albumentations.RandomBrightnessContrast / albumentations.GaussNoise / albumentations.ChannelShuffle /
albumentations.RandomResizedCrop / albumentations.HorizontalFlip / albumentations.Rotate /
etc.

4. arguments
albumentations.RandomBrightnessContrast(
    brightness_limit=0.2,   # 밝기 변화 범위 (-0.2~+0.2)
    contrast_limit=0.2,     # 대비 변화 범위 (-0.2~+0.2)
    p=0.5                   # 적용 확률
)
albumentations.GaussNoise(
    std_range=(0.1, 0.2),    # 표준편차 범위(이미지 픽셀 최대값을 기준으로 비율 지정)
    p=0.3                    # 적용 확률
)
albumentations.ChannelShuffle(
    p=0.5         # 적용 확률
)
albumentations.RandomResizedCrop(
    height=512,          # 출력 이미지 높이
    width=512,           # 출력 이미지 너비
    scale=(0.5, 0.9),    # 원본 이미지 대비 crop 영역 비율 - 지정안해도 기본값 적용됨
    ratio=(0.75, 1.33),  # crop 영역 가로/세로 비율 - 지정안해도 기본값 적용됨
    p=1.0                # 적용 확률
)
albumentations.HorizontalFlip(
    p=0.5         # 적용 확률
)
albumentations.Rotate(
    limit=30,     # 회전 각도 범위 (-30° ~ +30°)
    p=0.5         # 적용 확률
)
'''

def augment_train_data(train_ds, image_size, label_cnt, batch_size, shuffle=True):
    transforms = albumentations.Compose([
        albumentations.Normalize(),
        albumentations.RandomResizedCrop(
            size=(512, 512),
            scale=(0.5, 0.9),    # 원본 이미지 대비 crop 영역 비율 - 지정안해도 기본값 적용됨
            ratio=(0.75, 1.33),  # crop 영역 가로/세로 비율 - 지정안해도 기본값 적용됨
            p=1.0                # 적용 확률
        ),
        albumentations.HorizontalFlip(
            p=0.5         # 적용 확률
        ),
        albumentations.Rotate(
            limit=30,     # 회전 각도 범위 (-30° ~ +30°)
            p=0.5         # 적용 확률
        )    ])

    def aug_fn(image):
        data = {"image": image}
        aug_data = transforms(**data)
        aug_img = aug_data["image"]
        aug_img = tf.cast(aug_img, tf.float32)
        return aug_img

    def process_data(image, label):
        aug_img = tf.numpy_function(func=aug_fn, inp=[image], Tout=tf.float32)
        return aug_img, label

    def set_shapes(img, label, img_shape=image_size, label_count=label_cnt):
        img.set_shape(img_shape)
        label.set_shape(label_count)
        return img, label

    ds_alb = train_ds.map(partial(process_data), num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
    ds_alb = ds_alb.map(set_shapes, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # ds_alb = ds_alb.repeat()
    if shuffle:
        ds_alb = ds_alb.shuffle(buffer_size=batch_size * 10)
    ds_alb = ds_alb.batch(batch_size)
    return ds_alb


def preprocess_train_data(train_ds, image_size, label_cnt, batch_size, shuffle=True):
    transforms = albumentations.Compose([
        albumentations.Normalize(),
        albumentations.Resize(image_size[0], image_size[1]),
    ])

    def aug_fn(image):
        data = {"image": image}
        aug_data = transforms(**data)
        aug_img = aug_data["image"]
        aug_img = tf.cast(aug_img, tf.float32)
        return aug_img

    def process_data(image, label):
        aug_img = tf.numpy_function(func=aug_fn, inp=[image], Tout=tf.float32)
        return aug_img, label

    def set_shapes(img, label, img_shape=image_size, label_count=label_cnt):
        img.set_shape(img_shape)
        label.set_shape(label_count)
        return img, label

    ds_alb = train_ds.map(partial(process_data), num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
    ds_alb = ds_alb.map(set_shapes, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # ds_alb = ds_alb.repeat()
    if shuffle:
        ds_alb = ds_alb.shuffle(buffer_size=batch_size * 10)
    ds_alb = ds_alb.batch(batch_size)
    return ds_alb


def preprocess_val_data(val_ds, image_size, label_cnt, batch_size):
    transforms = albumentations.Compose([
        albumentations.Normalize(),
        albumentations.Resize(image_size[0], image_size[1]),
    ])

    def aug_fn(image):
        data = {"image": image}
        aug_data = transforms(**data)
        aug_img = aug_data["image"]
        aug_img = tf.cast(aug_img, tf.float32)
        return aug_img

    def process_data(image, label):
        aug_img = tf.numpy_function(func=aug_fn, inp=[image], Tout=tf.float32)
        return aug_img, label

    def set_shapes(img, label, img_shape=image_size, label_count=label_cnt):
        img.set_shape(img_shape)
        label.set_shape(label_count)
        return img, label

    ds_alb = val_ds.map(partial(process_data), num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
    ds_alb = ds_alb.map(set_shapes, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(batch_size)
    return ds_alb

In [ ]:
if __name__ == '__main__':
    tr_img, tr_class = place_data.parse_file_path(tr_img_root, class_dict)
    te_img, te_class = place_data.parse_file_path(te_img_root, class_dict)

    training_data = tf.data.Dataset.from_tensor_slices((tr_img, tr_class))
    validation_data = tf.data.Dataset.from_tensor_slices((te_img, te_class))

    training_data = training_data.map(place_data.load_image_and_label_from_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    validation_data = validation_data.map(place_data.load_image_and_label_from_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if use_augmentation:
        training_data = augment_train_data(training_data, image_size=img_size, label_cnt=len(class_dict.keys()),
                                           batch_size=batch_size, shuffle=True)

    else:
        training_data = preprocess_train_data(training_data, image_size=img_size, label_cnt=len(class_dict.keys()),
                                              batch_size=batch_size, shuffle=True)

    validation_data = preprocess_val_data(validation_data, image_size=img_size, label_cnt=len(class_dict.keys()),
                                          batch_size=batch_size)

    if use_pretrained_backbone:
        weights = "imagenet"

    else:
        weights = None

    x = tf.keras.layers.Input(shape=img_size)

    if backbone_type == "resnet50":
        feature = tfdet.model.backbone.resnet50(x, weights=weights)[-1]

    elif backbone_type == "resent101":
        feature = tfdet.model.backbone.resnet101(x, weights=weights)[-1]

    elif backbone_type == "vgg16":
        feature = tfdet.model.backbone.vgg16(x, weights=weights)[-1]

    elif backbone_type == "mobilenet":
        feature = tfdet.model.backbone.mobilenet_v2(x, weights=weights)[-1]

    else:
        # default : resnet50
        feature = tfdet.model.backbone.resnet50(x, weights=weights)[-1]

    out = place_classifier.Classifier(n_class=len(class_dict), n_feature=n_feature)(feature)
    model = tf.keras.Model(inputs=x, outputs=out)
    model.summary()

    os.makedirs("logs/{}/".format(output_name), exist_ok=True)
    os.makedirs("weight/{}/".format(output_name), exist_ok=True)

    logdir = "logs/{}/".format(output_name)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weight/{}/best_place_model.weights.h5".format(output_name),
                                                    monitor='val_accuracy', verbose=1,
                                                    save_best_only=True, save_weights_only=True, mode='max', save_freq='epoch')

    opt = tf.keras.optimizers.SGD(learning_rate, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(training_data, validation_data=validation_data, epochs=epochs, callbacks=[tensorboard_callback, checkpoint])
    model.save_weights("weight/{}/place_model.weights.h5".format(output_name))

100%|██████████| 8/8 [00:00<00:00, 652.30it/s]


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 512, 512, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 512, 512, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 512, 512, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 256, 256, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 256, 256, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 128, 128, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 128, 128, 256)  │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 128, 128, 256)  │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 64, 64, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 64, 64, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 64, 64, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 32, 32, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classifier_1 (Classifier)       │ (None, 8)              │       537,608 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,252,296 (58.18 MB)

 Trainable params: 15,250,248 (58.18 MB)

 Non-trainable params: 2,048 (8.00 KB)

Epoch 1/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6641 - loss: 1.6648
Epoch 1: val_accuracy improved from -inf to 0.55052, saving model to weight/vgg16_60_pretrained_x/best_place_model.weights.h5
121/121 ━━━━━━━━━━━━━━━━━━━━ 510s 4s/step - accuracy: 0.6647 - loss: 1.6603 - val_accuracy: 0.5505 - val_loss: 7.0044
Epoch 2/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4349 - loss: 3.3577
Epoch 2: val_accuracy did not improve from 0.55052
121/121 ━━━━━━━━━━━━━━━━━━━━ 185s 2s/step - accuracy: 0.4368 - loss: 3.3457 - val_accuracy: 0.5505 - val_loss: 2.2000
Epoch 3/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2576 - loss: 2.9123
Epoch 3: val_accuracy did not improve from 0.55052
121/121 ━━━━━━━━━━━━━━━━━━━━ 185s 2s/step - accuracy: 0.2596 - loss: 2.9050 - val_accuracy: 0.1505 - val_loss: 3.4479
Epoch 4/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2334 - loss: 1.8543
Epoch 4: val_accuracy did not improve from 0.55052
121/121 ━━━━━━━━━━━━━━━━━━━

In [ ]:
import tensorflow as tf
import keras
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.20.0
Keras version: 3.10.0
